In [1]:
!pip install shapely

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
import pandas as pd
import csv
from geopy.distance import geodesic
from shapely.geometry import Point
from shapely.ops import nearest_points


# 從 CSV 中讀取帶有緯度和經度欄位的商店資料
def read_coordinates_from_csv(file_path, lat_col, lon_col):
    coordinates = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            lat_str, lon_str = row[lat_col].strip(), row[lon_col].strip()
            if lat_str and lon_str and lat_str.replace('.', '').isdigit() and lon_str.replace('.', '').isdigit():
                lat, lon = float(lat_str), float(lon_str)
                coordinates.append(Point(lon, lat))  # 注意經緯度的順序
    return coordinates

# 讀取商店資料
stores = read_coordinates_from_csv('./外部資料/便利商店.csv', 'lat', 'lng')

#df_training = pd.read_csv('training_with_all_temp.csv')

# 定義計算附近商店數量的函數
def calculate_nearby_store_count(row):
    house_coords = Point(row['lon'], row['lat'])  # 注意經緯度的順序
    nearest_store = nearest_points(house_coords, stores)[1][0]  # 解決 numpy.ndarray 問題
    distance_km = geodesic((row['lat'], row['lon']), (nearest_store.y, nearest_store.x)).kilometers
    nearby_store_count = sum(1 for store_coords in stores if geodesic((row['lat'], row['lon']), (store_coords.y, store_coords.x)).kilometers <= 1)
    return nearby_store_count

# 使用 apply 函數新增 'nearby_store_count' 欄位
df_training['nearby_store_count'] = df_training.apply(calculate_nearby_store_count, axis=1)

# 顯示結果
print(df_training)

      Unnamed: 0        ID         lon        lat  house_age  \
0              0      TR-1  121.547608  25.022469  32.583333   
1              1      TR-2  121.502124  25.019127  24.166667   
2              2      TR-3  120.365799  22.640966   6.166667   
3              3      TR-4  121.462402  25.058664   8.833333   
4              4      TR-5  121.469444  25.023585  11.000000   
...          ...       ...         ...        ...        ...   
2470       11746  TR-11747  121.298684  24.935576  28.333333   
2471       11747  TR-11748  120.973622  24.795109  29.250000   
2472       11748  TR-11749  121.660968  25.072204  22.833333   
2473       11749  TR-11750  121.453610  24.982953  25.083333   
2474       11750  TR-11751  121.497284  25.057133   2.250000   

      residence_housing  congregate_housing  commercial_use  industrial_use  \
0                     1                   0               0               0   
1                     1                   0               0              

In [5]:
# 從 CSV 中讀取帶有緯度和經度欄位的商店資料
def read_coordinates_from_csv(file_path, lat_col, lon_col):
    coordinates = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            lat_str, lon_str = row[lat_col].strip(), row[lon_col].strip()
            if lat_str and lon_str and lat_str.replace('.', '').isdigit() and lon_str.replace('.', '').isdigit():
                lat, lon = float(lat_str), float(lon_str)
                coordinates.append(Point(lon, lat))  # 注意經緯度的順序
    return coordinates

# 讀取商店資料
stores = read_coordinates_from_csv('./外部資料/金融機構基本資料.csv', 'lat', 'lng')

#df_training = pd.read_csv('training_with_all_temp.csv')

# 定義計算附近商店數量的函數
def calculate_nearby_store_count(row):
    house_coords = Point(row['lon'], row['lat'])  # 注意經緯度的順序
    nearest_store = nearest_points(house_coords, stores)[1][0]  # 解決 numpy.ndarray 問題
    distance_km = geodesic((row['lat'], row['lon']), (nearest_store.y, nearest_store.x)).kilometers
    nearby_store_count = sum(1 for store_coords in stores if geodesic((row['lat'], row['lon']), (store_coords.y, store_coords.x)).kilometers <= 1)
    return nearby_store_count

# 使用 apply 函數新增 'nearby_store_count' 欄位
df_training['nearby_atm_count'] = df_training.apply(calculate_nearby_store_count, axis=1)

# 顯示結果
print(df_training)

      Unnamed: 0        ID         lon        lat  house_age  \
0              0      TR-1  121.547608  25.022469  32.583333   
1              1      TR-2  121.502124  25.019127  24.166667   
2              2      TR-3  120.365799  22.640966   6.166667   
3              3      TR-4  121.462402  25.058664   8.833333   
4              4      TR-5  121.469444  25.023585  11.000000   
...          ...       ...         ...        ...        ...   
2470       11746  TR-11747  121.298684  24.935576  28.333333   
2471       11747  TR-11748  120.973622  24.795109  29.250000   
2472       11748  TR-11749  121.660968  25.072204  22.833333   
2473       11749  TR-11750  121.453610  24.982953  25.083333   
2474       11750  TR-11751  121.497284  25.057133   2.250000   

      residence_housing  congregate_housing  commercial_use  industrial_use  \
0                     1                   0               0               0   
1                     1                   0               0              

In [ ]:
# 使用向量化操作計算距離
df_training['nearby_store_count'] = df_training.apply(lambda row: sum(np.linalg.norm(stores - [row['lon'], row['lat']], axis=1) <= 1), axis=1)

In [8]:
df_training = pd.read_csv('training_with_all_temp.csv')

In [7]:
df_training

,Unnamed: 0,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,...,total_junior,d_junior,total_hight,d_high,total_univr,d_univr,d_temple,temple_distance_less_than_500m,nearby_store_count,nearby_atm_count
0,0,TR-1,121.547608,25.022469,32.583333,1,0,0,0,0,...,533,0.3518,1386,0.3518,5938,0.2498,0.047350,True,56,38
1,1,TR-2,121.502124,25.019127,24.166667,1,0,0,0,0,...,445,1.0811,1709,1.1032,8114,2.1874,0.334299,True,32,4
2,2,TR-3,120.365799,22.640966,6.166667,0,1,0,0,0,...,700,1.5075,2106,1.1164,17568,2.0588,0.842736,False,15,2
3,3,TR-4,121.462402,25.058664,8.833333,0,1,0,0,0,...,1051,0.5736,1678,1.4718,25760,4.0438,0.348776,True,44,15
4,4,TR-5,121.469444,25.023585,11.000000,1,0,0,0,0,...,1357,0.5955,648,0.9198,11486,0.5154,0.200054,True,98,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,11746,TR-11747,121.298684,24.935576,28.333333,1,0,0,0,1,...,1069,1.3890,5378,5.1675,2762,4.7411,0.731255,False,13,2
2471,11747,TR-11748,120.973622,24.795109,29.250000,1,0,0,0,0,...,576,0.3291,1921,0.6855,17488,2.3293,0.238710,True,29,9
2472,11748,TR-11749,121.660968,25.072204,22.833333,1,0,0,0,0,...,589,0.6408,924,0.7140,4850,9.5361,0.309986,True,34,6
2473,11749,TR-11750,121.453610,24.982953,25.083333,1,0,0,0,0,...,1029,1.1976,2593,0.2405,4996,1.2017,0.260047,True,52,13


In [8]:
df_training.to_csv('training_ALLdata_plus_numcon.csv')